In [ ]:
import itertools

import networkx as nx
from networkx.algorithms import bipartite

import pandas as pd

In [ ]:
tags = pd.Series.from_csv("/tmp/out/tags.csv")

In [ ]:
tags = tags.apply(lambda x: eval(x))

In [ ]:
tags[0]

In [ ]:
# [item for sublist in l for item in sublist]

tags_all = set(list(itertools.chain(*tags)))

len(tags_all)

In [ ]:
g = nx.Graph()

g.add_nodes_from(list(tags.index), bipartite=0)
g.add_nodes_from(tags_all, bipartite=1)

for i,v in enumerate(tags):
    g.add_edges_from([ (i,t) for t in v ])

In [ ]:
nx.info(g)

In [ ]:
g2 = bipartite.weighted_projected_graph(g, tags_all)

In [ ]:
nx.info(g2)

In [ ]:
import holoviews as hv

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from datashader.layout import random_layout, circular_layout, forceatlas2_layout
from datashader.bundling import connect_edges, hammer_bundle

In [ ]:
layout = nx.circular_layout(g2)

In [ ]:
data = [[node]+layout[node].tolist() for node in g2.nodes]

nodes = pd.DataFrame(data, columns=['id', 'x', 'y'])
nodes.set_index('id', inplace=True)

edges = pd.DataFrame(list(g2.edges), columns=['source', 'target'])

In [ ]:
bundled = hammer_bundle(nodes, edges)

In [ ]:
cvsopts = dict(plot_height=400, plot_width=400)

xr = nodes.x.min(), nodes.x.max()
yr = nodes.y.min(), nodes.y.max()

canvas = ds.Canvas(x_range=xr, y_range=yr, **cvsopts)

aggregator=None
agg=canvas.points(nodes,'x','y',aggregator)

np = tf.spread(tf.shade(agg, cmap=["#FF3333"]), px=3)
ep = tf.shade(canvas.line(bundled, 'x','y', agg=ds.count()))

img = tf.stack(ep, np, how="over")

img

In [ ]:
ds.utils.export_image(img=img, filename="tags_bundled.png", fmt=".png", background="white")